# Sarcoma Data Analysis
In this notebook, we apply our proposed method and comparators to the sarcoma dataset generated by `data_cleaning.ipynb`.

In [ ]:
# Basic Imports
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse

# sklearn imports
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.linear_model import LogisticRegression as LR
from sklearn.naive_bayes import GaussianNB as GNB

# Import statsmodels for mixed models
import statsmodels.formula.api as smf

# Sarcoma Dataset

In [ ]:
# Read in data
data = pd.read_csv("../data/combined_sarcoma_data.csv")

# Clean column names
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('-', '_')

# Change MALE to 0, FEMALE to 1
data = data.replace({"MALE": 0, "FEMALE":1})

# Drop any rows with missing data
data = data.dropna(axis=0)

# Normalize the data
data['Y'] = (data['Y'] - data['Y'].mean()) / data['Y'].std()

# Train-test split
X_train = data[~data["short_histo"].isin(['SS', 'MPNST'])]

X_test = data[data["short_histo"].isin(['SS', 'MPNST'])]
# X_test = data[data["short_histo"].isin(['SS'])]
# X_test = data[data["short_histo"].isin(['MPNST'])]

features = ['age_at_diagnosis', 'gender', 'JUN',
       'VGLL3', 'TERT', 'MAP3K5', 'UST', 'CDKN2A', 'YAP1', 'CDKN1B', 'PTPRQ',
       'RB1', 'TP53', 'MYOCD', 'NF1', 'CCNE1', 'CEBPA', 'ZNF552', 'ATRX',
       'PTEN', 'DDIT3', 'CDK4', 'HMGA2', 'MDM2', 'FRS2', 'Silent_per_Mb',
       'Non_silent_per_Mb', 'CD274', 'CTLA4', 'HAVCR2', 'LAG3', 'PDCD1', 
       'TCF7', 'TIGIT']

output = 'Y'

In [ ]:
# Regular Tree

performances = []

for i in range(100):
    tree = DTR(random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    tree_mse = mse(X_test[output], pred)
    performances.append(tree_mse)

print("Tree MSE:", np.mean(performances))

In [ ]:
# Regular random forest

performances = []

for i in range(100):
    tree = RFR(n_estimators=5, random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    forest_mse = mse(X_test[output], pred)
    performances.append(forest_mse)

print("Forest (5 Trees) MSE:", np.mean(performances))

In [ ]:
# Regular random forest

performances = []

for i in range(20):
    tree = RFR(n_estimators=100, random_state=i)
    tree.fit(X_train[features], X_train[output])
    pred = tree.predict(X_test[features])
    forest_mse = mse(X_test[output], pred)
    performances.append(forest_mse)

print("Forest (100 Trees) MSE:", np.mean(performances))

In [ ]:
# Build group classifier
group_clf = LR()  # GNB() or DTC() or RFC() or LR()
group_clf.fit(X_train[features], X_train['short_histo'])
group_pred = group_clf.predict_proba(X_test[features])

print("Order of classes: ", group_clf.classes_)

# Normalize group predictions
row_sums = group_pred.sum(axis=1)
group_pred = group_pred / row_sums[:, np.newaxis]

for test_group in ['SS', 'MPNST']:
    rows = np.where(X_test['short_histo'] == test_group)
    average = np.mean(group_pred[rows,], axis=1)
    group_pred[rows] = average

In [ ]:
# Mixture of Trees

performances = []

for i in range(100):

    list_of_trees = []
    for group in ['DDLPS', 'MFS', 'STLMS', 'ULMS', 'UPS']:
        tree = DTR(random_state=i)
        tree.fit(
            X_train[X_train["short_histo"] == group][
                features
            ],
            X_train[X_train["short_histo"] == group][output],
        )
        list_of_trees.append(tree)

    preds = np.array(
        [
            tree.predict(X_test[features])
            for tree in list_of_trees
        ]
    )
    preds = preds.T
    num = preds.shape[0]
    pred = [np.dot(preds[i, :], group_pred[i, :]) for i in range(num)]
    performances.append(mse(X_test[output], pred))



print("Mixture of Trees MSE:", np.mean(performances))

In [ ]:
# Mixture of RF

performances = []

for i in range(100):

    list_of_trees = []
    for group in ['DDLPS', 'MFS', 'STLMS', 'ULMS', 'UPS']:
        tree = RFR(n_estimators=20, random_state=i)
        tree.fit(
            X_train[X_train["short_histo"] == group][
                features
            ],
            X_train[X_train["short_histo"] == group][output],
        )
        list_of_trees.append(tree)

    preds = np.array(
        [
            tree.predict(X_test[features])
            for tree in list_of_trees
        ]
    )
    preds = preds.T
    num = preds.shape[0]
    pred = [np.dot(preds[i, :], group_pred[i, :]) for i in range(num)]
    performances.append(mse(X_test[output], pred))



print("Mixture of 20 Tree Forest MSE:", np.mean(performances))

In [ ]:
formula = " ~ " + " + ".join(features)
output  + formula

In [ ]:
# LMM

formula = " ~ " + " + ".join(features)

md = smf.mixedlm(output + formula, X_train, groups=X_train["short_histo"])
mdf = md.fit()
pred = mdf.predict(X_test)
lmm_mse = mse(X_test[output], pred)
print("LMM MSE:", lmm_mse)
